## Data preparation - sketch to pixel art (Model A3)

### Create pairs of sketches and pixel art images

- TinyHero dataset source: https://www.kaggle.com/datasets/calmness/retro-pixel-characters-generator
- Data for transfer learning:
    - Anime Faces dataset source: https://www.kaggle.com/datasets/soumikrakshit/anime-faces
    - Pokemon data source: https://www.kaggle.com/datasets/zackseliger/pokemon-images-includes-fakemon
    - Animal Pack game assets source: https://kenney.nl/assets/animal-pack-redux
- Data preparation for model A3 uses data from the `datasets/unprepared_data/input_a3` folder.
- Prepared data is saved in `datasets/model_a3_data`
- Pix2Pix models requires input images and ground truth images to be combined. These prepared images are stored in the `combined` folder(s) in `datasets/model_a3_data`

In [1]:
import os, glob, random
import common_functions as core

# Get path for files needed to prepare data for transfer learning (base model A3)
TRANSFER_LEARNING_DATA = core.get_path(core.A3_BASE.raw_data_dir)
# Get path for files needed to prepare data for model A3
CHARACTER_DATA = core.get_path(core.A3.raw_data_dir)

# File paths to save prepared transfer learning data to train base model A3
TL_INPUT_DATA = core.get_path(core.A3_BASE.prepared_data_dir, f"{core.A3_BASE.dataset_path_prefix}input")
TL_OUTPUT_DATA = core.get_path(core.A3_BASE.prepared_data_dir, f"{core.A3_BASE.dataset_path_prefix}output")
TL_COMBINED_DATA = core.get_path(core.A3_BASE.prepared_data_dir, f"{core.A3_BASE.dataset_path_prefix}combined")

# File paths to save prepared data for training model A3
PIX2PIX_INPUT_DATA = core.get_path(core.A3.prepared_data_dir, f"{core.A3.dataset_path_prefix}input")
PIX2PIX_OUTPUT_DATA = core.get_path(core.A3.prepared_data_dir, f"{core.A3.dataset_path_prefix}output")
PIX2PIX_COMBINED_DATA = core.get_path(core.A3.prepared_data_dir, f"{core.A3.dataset_path_prefix}combined")

# Set random seed for reproducibility
random.seed(187)

# Sorted list of images needed to prepare transfer learning data (data for base model A3)
transfer_learn_images = sorted(list(glob.glob(TRANSFER_LEARNING_DATA + "/*.png")))
# Sorted list of TinyHero images to prepare data for model A3
character_images = sorted(list(glob.glob(CHARACTER_DATA + "/*.png")))

# Shuffle lists of images
random.shuffle(transfer_learn_images)
random.shuffle(character_images)

# Folder names to store train, test, validation data
split_folders = ("train", "test", "val")

## Folder structure for model A3 data folder

```
model_a3_data
    # Prepared data for Pix2Pix model
    -- pix2pix_input (input images)
        -- train
        -- test
        -- val
    -- pix2pix_output (target images)
        -- train
        -- test
        -- val
    -- pix2pix_combined (combined input, target image pairs to use for Pix2Pix training)
        -- train
        -- test
        -- val

    # Prepared data for transfer learning
    -- tl_input (input images)
            -- train
            -- test
            -- val
    -- tl_output (target images)
            -- train
            -- test
            -- val
    -- tl_combined (combined input, target image pairs to use for Pix2Pix training)
            -- train
            -- test
            -- val
```

In [2]:
def prepare_data(input_images, model_inp_dir, model_out_dir, split = (0.8, 0.9, 1.0)):
    """
    Function to prepare data folders
    :param input_images: original images
    :param model_inp_dir: folder for storing prepared sketches
    :param model_out_dir: folder for storing target images (ground truth images)
    :param split: split points for train, test, validation
    """
    counter = 0
    sketch_counter = 0
    # Get split point for train data (0 -> train_end)
    train_end = int(len(input_images) * split[0])
    # Get split point for test data (train_end -> test_end)
    # Validation data is from test_end -> end
    test_end = int(len(input_images) * split[1])
    
    # 3 sketches will be created per image
    sketch_per_img = 3
    
    # Outer for loop to iterate through input images
    for image in input_images:
        # Load and resize image with magenta background to 64x64
        # then convert to OpenCV image
        magenta_bg_image = core.pil_to_opencv(core.load_64x64_with_magenta_bg(image))
        
        # Inner for loop to iterate through sketches created per image
        for sketch in range(1, sketch_per_img + 1):
            # Create sketch using levels=sketch parameter to decide the level of detail in the sketch
            # The higher the 'levels' parameter -> smoother the sketching
            sketch_image = core.create_sketch(magenta_bg_image, levels=sketch, magenta_bg=True)
            # Zero padded file names
            filename = f"{sketch_counter:05}.png"
            # Set 'current_split_folder' to train data folder
            current_split_folder = split_folders[0]
            # Then set 'current_split_folder' based on 'counter' variable
            if train_end < counter < test_end:
                current_split_folder = split_folders[1] # test data folder
            elif counter >= test_end:
                current_split_folder = split_folders[2] # validation data folder
            
            # ----------- Write to model_input and output directories -----------
            model_input_path = os.path.join(model_inp_dir, current_split_folder)
            os.makedirs(model_input_path, exist_ok=True)
            core.write_image(sketch_image, os.path.join(model_input_path, filename))
            
            model_out_path = os.path.join(model_out_dir, current_split_folder)
            os.makedirs(model_out_path, exist_ok=True)
            core.write_image(magenta_bg_image, os.path.join(model_out_path, filename))
            sketch_counter += 1
        counter += 1

In [3]:
# Prepare data for transfer learning Pix2Pix (A3 base model)
prepare_data(transfer_learn_images, TL_INPUT_DATA, TL_OUTPUT_DATA)

In [4]:
# Prepare data for final A3 Pix2Pix
prepare_data(character_images, PIX2PIX_INPUT_DATA, PIX2PIX_OUTPUT_DATA)

In [5]:
# Create combined images of input and ground truth images needed for transfer learning (base A3 model)
os.makedirs(TL_COMBINED_DATA, exist_ok=True)
core.create_combined_images(f"--fold_A {TL_INPUT_DATA} --fold_B {TL_OUTPUT_DATA} --fold_AB {TL_COMBINED_DATA} --no_multiprocessing")

----------------------------------------------------------
python /Users/tashvit/Documents/GitHub/mmpixagen/thirdparty/pix2pix/datasets/combine_A_and_B.py --fold_A /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_input --fold_B /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_output --fold_AB /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_combined --no_multiprocessing
[fold_A] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_input
[fold_B] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_output
[fold_AB] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/tl_combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 1014/1014 images
split = test, number of images = 1014
split = train, use 8127/8127 images
split = train, number of images = 8127
split = val, use 1017/1017 images
split = val, number of images = 1017


In [6]:
# Create combined images of input and ground truth images for final A3 model
os.makedirs(PIX2PIX_COMBINED_DATA, exist_ok=True)
core.create_combined_images(f"--fold_A {PIX2PIX_INPUT_DATA} --fold_B {PIX2PIX_OUTPUT_DATA} --fold_AB {PIX2PIX_COMBINED_DATA} --no_multiprocessing")

----------------------------------------------------------
python /Users/tashvit/Documents/GitHub/mmpixagen/thirdparty/pix2pix/datasets/combine_A_and_B.py --fold_A /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_input --fold_B /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_output --fold_AB /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_combined --no_multiprocessing
[fold_A] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_input
[fold_B] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_output
[fold_AB] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_a3_data/pix2pix_combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 270/270 images
split = test, number of images = 270
split = train, use 2190/2190 images
split = train, number of images = 2190
split = val, use 276/276 images
split = val, number of images =